## Creating processes and Importing Packages

In [2]:
using MPIClusterManagers
using Distributed
if procs() == workers()
    nranks = 5
    manager = MPIWorkerManager(nranks)
    addprocs(manager)
end

5-element Vector{Int64}:
 2
 3
 4
 5
 6

In [3]:
@everywhere workers() begin 
    using Random
    using Distributions
    using CSV
    using DataFrames
end

In [107]:
@everywhere workers() begin
    using MPI
    MPI.Init()
    comm = MPI.COMM_WORLD
    nranks = MPI.Comm_size(comm)
    rank = MPI.Comm_rank(comm)
    println("Hello, I am process $rank of $nranks processes!")
end

      From worker 2:	Hello, I am process 0 of 5 processes!
      From worker 3:	Hello, I am process 1 of 5 processes!
      From worker 4:	Hello, I am process 2 of 5 processes!
      From worker 5:	Hello, I am process 3 of 5 processes!
      From worker 6:	Hello, I am process 4 of 5 processes!


## Question 3

### General functions

In [7]:
## GEneral funtions
@everywhere function mandelbrot_set(c,iters)
    "Does n iteration for the mandelbor set for c"
    z = 0 # Mandelbrot set start always from 0
    n = 0
    while abs(z) <= 2 && n<iters
        z = z^2 + c
        n += 1
    end
    return n
end

@everywhere function area_general(det)
    "Function that given a list of 0 and 1 of the mandelbrot set, it returns the area of 0 over 1 for an area of 6"
    area_tot = 6
    area_not = length(det[det .== 0.])
    area_in = length(det[det .== 1.])
    
    area = area_tot*area_in/(area_not+area_in) ## Note, we compute 6*area_in/tot bu floating point
    
    return area
end

@everywhere function detector(c,iters)
    "Functions that return 1 if a number is in mandelbrot set, and 0 if it is not"
    n = mandelbrot_set(c,iters)
    if n == iters ## If the number is inside we get 1
        return 1
    else ## If the number is outside we get 0
        return 0
    end
end

### Mandel pure functions

In [6]:

@everywhere function area_mandel_pure(s,iters)

    "Given a sample of numbers and a maximum iterations iters, it return the area computed with mandel_pure"
    det = Int64[]
    for i in 1:s
        x = rand(Uniform(-2,1))
        y = rand(Uniform(-1,1))
        c = Complex(x,y)
        det_s = detector(c,iters)
        push!(det,det_s)
    end
    area = area_general(det)
    return area
end

In [7]:
@everywhere workers() begin
    area_mandel_pure(100,100)
end

### Latin Functions

#### Sub functions

In [8]:
@everywhere function latin_num_gen(s,lim_x,lim_y)
    "Function that generates the latin numbers, lim_x and lim_y are the max, and min values of x and y (x_min,x_max)"
    
    spacing_x = (lim_x[2] - lim_x[1])/s
    spacing_y = (lim_y[2] - lim_y[1])/s
    c_x = Float64[]
    c_y = Float64[]
    
    for i in 0:(s-1)
        push!(c_x,rand(Uniform(lim_x[1] + spacing_x*i,lim_x[1] + spacing_x*(1+i))))
        push!(c_y,rand(Uniform(lim_y[1] + spacing_y*i,lim_y[1] + spacing_y*(1+i))))
    end

    shuffle!(c_x)
    shuffle!(c_y)

    c = complex.(c_x,c_y)
    
    return c
end


@everywhere function latin_num_det(c,iters)
    "It returns an array with all the det of a subspace, for latin hypercube the subspace is all the space"
    z = 0
    n = 0
    while n < iters
        z = z.^2 .+ c
        n += 1
    end
    return z
end



#### Function


In [9]:
@everywhere function area_mandel_latin(s,iters)
    
    "Do the latin hypercubes for a sample size in monterblot"
    lim_x = (-2,1)
    lim_y = (-1,1)

    c = latin_num_gen(s,lim_x, lim_y)
    
    det = abs.(latin_num_det(c,iters))

    det_in = filter(x -> x ≤ 2, det) 

    area = 6*(length(det_in))/s

    return area
end


In [10]:
@everywhere workers() begin
    a = area_mandel_latin(400,200)
    println(a)
end

      From worker 6:	1.38
      From worker 5:	1.515
      From worker 4:	1.695


      From worker 3:	1.59
      From worker 2:	1.5


### Ortoghonal Functions


#### Sub functions


In [103]:
@everywhere function intervals_gen(s)
    "Generates al the intervals for orthogoanl sampling"
    spacing_x = (3)/s
    spacing_y = (2)/s
    intervals_x = hcat(collect(-2:spacing_x:1-spacing_x), collect(-2+spacing_x:spacing_x:1))
    intervals_y = hcat(collect(-1:spacing_y:1-spacing_y), collect(-1+spacing_y:spacing_y:1))
    return intervals_x, intervals_y

end

@everywhere function  orto_num_gen(s_i, lim_x, lim_y, intervals_x, intervals_y)
    "Function that given a subspace, it creates the values on it"
    s_i = Int(s_i)
    fil_intervals_x = intervals_x[(intervals_x[:, 1].>=lim_x[1]).&(intervals_x[:, 2].<=lim_x[2]),:]
    
    fil_intervals_y = intervals_y[(intervals_y[:, 1].>=lim_y[1]).&(intervals_y[:, 2].<=lim_y[2]),:]
    cols_n = randperm(size(fil_intervals_x, 1))[1:s_i]
    rows_n = randperm(size(fil_intervals_y, 1))[1:s_i]

    cols = fil_intervals_x[cols_n,:]
    rows = fil_intervals_y[rows_n,:]
    
    c_x = [rand(Uniform(a,b)) for (a, b) in eachrow(cols)]
    c_y = [rand(Uniform(a,b)) for (a, b) in eachrow(rows)]
    
    for (col,row) in zip(eachrow(cols),eachrow(rows))

        col = hcat(col...)
        row = hcat(row...)

        col_del = findall(all(intervals_x .== col, dims = 2))[1][1]
        intervals_x = vcat(intervals_x[1:col_del-1,:], intervals_x[col_del+1:size(intervals_x)[1], :])

        row_del = findall(all(intervals_y .== row, dims = 2))[1][1]
        intervals_y = vcat( intervals_y[1:row_del-1,:], intervals_y[row_del+1:size(intervals_y)[1], :])
    end

    shuffle!(c_x)
    shuffle!(c_y)
    c = hcat(c_x,c_y)

    return c, intervals_x, intervals_y
end

#### Function from Julia

In [104]:
@everywhere function area_mandel_ort(s,iters)
    "We generate x subspaces, and each subspace we do latin.The number of samples must be multiple of subspaces"
    a = time()
    n_subspaces = 10
    n_rows = 2
    n_cols = 5
    intervals_x, intervals_y = intervals_gen(s)
    intervals_x_np, intervals_y_np = intervals_x, intervals_y
    det = Float64[]
    subspace_x = Int(length(intervals_x)/(2*n_cols)) # Intervals of each subspace
    subspace_y = Int(length(intervals_y)/(2*n_rows))
    s_i = s/n_subspaces ## Number of values to compute in each subspace
    c_l = Matrix{Float64}(undef,0,2) 
   
    for i in 0: n_cols-1 # We get the limits for each subspace and compute the latin numbers

        low_bound_x = trunc(Int,subspace_x*i + 1)
        upper_bound_x = trunc(Int,subspace_x*i + subspace_x)
        lim_x = (intervals_x_np[low_bound_x,1],intervals_x_np[upper_bound_x,2])

        for j in 0:n_rows-1
            
            lim_y = (intervals_y_np[trunc(Int,subspace_y*j+1),1],intervals_y_np[trunc(Int,subspace_y*j + subspace_y),2])
            c,intervals_x,intervals_y = orto_num_gen(s_i,lim_x,lim_y, intervals_x, intervals_y)
            c_l = vcat(c_l, c)
        end
    end

    c_l = complex.(c_l[:, 1], c_l[:, 2])
    det = abs.(latin_num_det(c_l,iters))
    det_in = filter(x -> x ≤ 2, det) 
    area = 6*(length(det_in))/s

    return area
end

In [16]:
@everywhere workers() begin
    a = area_mandel_ort(200,200)
    println(a)
end

      From worker 3:	1.53
      From worker 4:	1.26
      From worker 5:	1.32
      From worker 2:	1.44
      From worker 6:	1.59


## Simulations

### Functions


In [127]:
@everywhere function n_simulations(model,n,samples)
    iterations = 200
    dict = Dict()
    for sample in samples
        areas = Float64[]
        for sim in 0:n
            area = model(sample,iterations)
            push!(areas,area)
        end
        dict["$sample"] = areas
    end
    return dict
end

@everywhere function question_3(model, max_samples, n,rank,nranks)
    samples = collect(100:100:max_samples)
    df = n_simulations(model,n,samples) 
    df = DataFrame(df)

    if model == area_mandel_latin
        CSV.write("Latin by $rank with $n simulations and 2000 samples .csv",df)
    end
    
    if model == area_mandel_pure
        CSV.write("Pure by $rank with $n simulations and 2000.csv",df)
    end
   
    if model == area_mandel_ort
        CSV.write("Ort by $rank with $n simulations and 2000.csv",df)
    end
end

### Simulations

In [128]:
@everywhere workers() begin
    using MPI
    MPI.Init()
    comm = MPI.COMM_WORLD
    nranks = MPI.Comm_size(comm)
    rank = MPI.Comm_rank(comm)
    question_3(area_mandel_pure,200,10,rank,nranks)
    question_3(area_mandel_latin,200,10,rank,nranks)
    question_3(area_mandel_ort,200,10,rank,nranks)
end

In [94]:
@everywhere workers() begin
    using MPI
    MPI.Init()
    comm = MPI.COMM_WORLD
    nranks = MPI.Comm_size(comm)
    rank = MPI.Comm_rank(comm)
    question_3(area_mandel_pure,2000, 2000,rank,nranks)
    question_3(area_mandel_latin,2000, 2000,rank,nranks)
    question_3(area_mandel_ort,2000, 2000,rank,nranks)
end